In [24]:
import numpy as np
import pandas as pd
import pickle

In [2]:
books = pd.read_csv('Books.csv')
# books = pd.read_csv('Books.csv', sep = ";", error_bad_lines = False, encoding = "latin-1")
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

<ipython-input-2-6a43c0b645e8>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [3]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [4]:
print(books.columns)
print(ratings.columns)
print(users.columns)

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')
Index(['User-ID', 'Location', 'Age'], dtype='object')


In [3]:
books.rename(columns = {'Book-Title': 'title', 'Book-Author': 'author', 'Year-Of-Publication': 'year', 'Publisher': 'publisher', 'Image-URL-M': 'imageM'}, inplace = True)
books.head(1)

,ISBN,title,author,year,publisher,Image-URL-S,imageM,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...


In [4]:
ratings.rename(columns = {'User-ID': 'user_ID', 'Book-Rating': 'rating'}, inplace = True)
ratings.head(1)

,user_ID,ISBN,rating
0,276725,034545104X,0


In [5]:
users.rename(columns = {'User-ID': 'user_ID'}, inplace = True)
users.head(1)

,user_ID,Location,Age
0,1,"nyc, new york, usa",NaN


In [16]:
print(books.isnull().sum())
print(ratings.isnull().sum())
print(users.isnull().sum())

ISBN           0
title          0
author         2
year           0
publisher      2
Image-URL-S    0
imageM         0
Image-URL-L    3
dtype: int64
user_ID    0
ISBN       0
raing      0
dtype: int64
user_ID          0
Location         0
Age         110762
dtype: int64


In [6]:
books.dropna(inplace = True)

In [7]:
x = ratings['user_ID'].value_counts() > 200
y = x[x].index
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_ID', length=899)

In [8]:
ratings = ratings[ratings['user_ID'].isin(y)]
ratings

,user_ID,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [9]:
ratings_with_books = ratings.merge(books, on = 'ISBN')
print(ratings_with_books['ISBN'].nunique())
print(ratings_with_books.shape)

178012
(487665, 10)


In [10]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns = {'rating': 'no_of_ratings'}, inplace = True)
number_rating.sort_values(by = 'no_of_ratings', ascending = False)

,title,no_of_ratings
156308,Wild Animus,363
19139,Bridget Jones's Diary,277
130220,The Lovely Bones: A Novel,270
132887,The Notebook,241
133894,The Pelican Brief,236
...,...,...
93079,Portrait sÃ?Â©pia,1
93078,Portrait of the Theatre,1
93077,Portrait of the Psychopath as a Young Woman,1
38036,Edge of Violence,1


In [11]:
final_rating = ratings_with_books.merge(number_rating, on = 'title')
final_rating.head(1)

,user_ID,ISBN,rating,title,author,year,publisher,Image-URL-S,imageM,Image-URL-L,no_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,82


In [12]:
final_rating = final_rating[final_rating['no_of_ratings'] >= 50]
final_rating.shape # those books whose min no of ratings is 50 as well as those each customers did min of 200 ratings

(61853, 11)

In [13]:
final_rating.drop_duplicates(['user_ID', 'title'], inplace = True)
final_rating.shape

(59850, 11)

In [14]:
book_pivot = final_rating.pivot_table(columns = 'user_ID', index = 'title', values = 'rating')
book_pivot.fillna(0, inplace = True)
book_pivot

user_ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from scipy.sparse import csr_matrix
book_sparce = csr_matrix(book_pivot)

In [16]:
type(book_sparce)

scipy.sparse._csr.csr_matrix

In [17]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm = 'brute')
model.fit(book_sparce)

NearestNeighbors(algorithm='brute')

In [18]:
distances, suggestions = model.kneighbors(book_pivot.iloc[240, :].values.reshape(1, -1), n_neighbors = 6)

In [19]:
book_pivot.index[237]

'Harry Potter and the Chamber of Secrets (Book 2)'

In [20]:
suggestions

array([[240, 238, 237, 241, 239, 184]])

In [21]:
# for i in range(len(suggestions)):
#   print(book_pivot.index[suggestions[i]])


for i in range(len(suggestions)):
  x = book_pivot.index[suggestions[i]]
  for i in range(len(x)):
    print(x[i])

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Order of the Phoenix (Book 5)
Exclusive


In [22]:
def recommend_book(book_name):
  book_id = np.where(book_pivot.index==book_name)[0][0]
  distances, suggestions = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors = 6)

  for i in range(len(suggestions)):
    x = book_pivot.index[suggestions[i]]
    for i in range(len(x)):
      if x[i] == book_name:
        continue
      else:
        print(x[i])

In [29]:
recommend_book('Harry Potter and the Prisoner of Azkaban (Book 3)')

Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Sorcerer's Stone (Book 1)
Harry Potter and the Order of the Phoenix (Book 5)
Exclusive


In [26]:
pickle.dump(final_rating,open('final_rating.pkl','wb'))
pickle.dump(model,open('model.pkl','wb'))
pickle.dump(book_pivot,open('book_pivot.pkl','wb'))